In [10]:
import pandas as pd
import numpy as np
import os

from itertools import product

import Finalized_pipeline

import seaborn as sns
import matplotlib.pyplot as plt

In [11]:
main_datasets = ["BACE_class", "BACE_regre", "ROR_class", "ROR_regre"]
model_name_shorts = {"DecisionTreeRegressor": "dt", "DecisionTreeClassifier": "dt", 
                    "RandomForestRegressor": "rf", "RandomForestClassifier": "rf", 
                    "LinearRegression": "lr", "LogisticRegression": "lr", 
                    "MLPRegressor": "nn", "MLPClassifier": "nn",  
                    "GradientBoostingRegressor": "gb", "GradientBoostingClassifier": "gb",
                    "XGBRegressor": "xg", "XGBClassifier": "xg", 
                    "SVR": "sv", "SVC": "sv"}
    
top_models = 100
split = 0.8


In [12]:
for dataset in main_datasets:
    regression = False
    if "regre" in dataset:
        regression = True


    df = pd.read_csv(r"Merged runs/" + dataset + ".csv")
    
    setups = {"Fingerprints": [], "Descriptors": [], "DescFing": []}

    for row in df.head(top_models)[['hyperparams', 'preprocessing', 'model']].iterrows():
        #print(row)
        params = list(row[1])[0].split(", '")
        params[0] = params[0][2:]
        params[-1] = params[-1][:-1]
        #print(params)

        preprocessing = list(row[1])[1]
        model = list(row[1])[2]
        #print(preprocessing)

        clean_params = {}
        
        for param in params:
            name = param.split("': ")[0]
            val = param.split("': ")[1]
            if val[0] == "'":
                val = val[1:-1]
            elif val[0] == "(":
                if val == "(50, 50)": val = (50, 50)
                if val == "(100, )" or val == "(100,)": val = (100, )
                if val == "(50, )" or val == "(50,)": val = (50, )
            elif val == "None":
                val = None
            else:
                try:
                    val = int(val)
                except:
                    val = float(val)
            clean_params[name] = [val]
        #print(model)
        #print(preprocessing)
        #print(clean_params)
        setups[preprocessing].append([model, clean_params])

    for key in setups.keys():
        setups[key] = sorted(setups[key], key=lambda x:x[0])
    #print(setups)

    returned_df = pd.DataFrame(columns=df.columns)
    #print(returned_df)

    for key in setups.keys():
        if key == "Descriptors":
            calculate_descriptors = True
            calculate_fingerprints = False
        if name == "DescFing":
            calculate_descriptors = True
            calculate_fingerprints = True
        if key == "Fingerprints":
            calculate_descriptors = False
            calculate_fingerprints = True

            
        if "BACE" in dataset:
            prepared_dataset = pd.read_csv(r"C:\Users\wojci\Documents\GitHub\czasteczkowa-inzynierka\experiments\split_datasets\split" + str(split) + "_bace.csv")
        else:
            prepared_dataset = pd.read_csv(r"C:\Users\wojci\Documents\GitHub\czasteczkowa-inzynierka\experiments\split_datasets\split" + str(split) + "_ROR_data_1.csv")
        
        if not "pIC50" in prepared_dataset.columns:
            prepared_dataset = Finalized_pipeline.calculate_pIC50(prepared_dataset, "target")
        
        if not regression:
            prepared_dataset = Finalized_pipeline.calculate_classification_labels(prepared_dataset, "pIC50", threshold=7)
            target_column = "label"
        else:
            target_column = "pIC50"

        if "mol" in prepared_dataset.columns:
            prepared_dataset.rename(columns={"mol": "SMILES"}, inplace=True)

        if regression:
            runtype = "regression"
        else:
            runtype = "classification"
        
        prepared_dataset = Finalized_pipeline.calculate_features(prepared_dataset, calculate_descriptors=calculate_descriptors, 
                                                                 calculate_fingerprints=calculate_fingerprints, 
                                                    SMILES_column_name="SMILES", target_column_name=target_column, 
                                                    split_column_name="Split", output_csv_path=dataset + "_" + key + "_" + str(split) + ".csv")

        for combination in setups[key]:
                            # model short name, parameters
            hyperparams = {str(model_name_shorts[combination[0]]): combination[1]}


            new_df = Finalized_pipeline.hyperparameter_search(prepared_dataset, hyperparams, unique=False, return_df=True, output_file_name="tmp.csv")

            new_df['split'] = split
            new_df['preprocessing'] = key

            returned_df = pd.concat([returned_df, new_df])

    if "mse" in returned_df.columns:
        returned_df.sort_values(by=['mse'], inplace=True)
    else:
        returned_df.sort_values(by=['roc_auc'], inplace=True, ascending=False)
    returned_df.to_csv("RERUN_" + dataset + ".csv", index=False)




A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Regression:  False
gb
{'max_depth': 5, 'loss': 'exponential', 'n_estimators': 10, 'learning_rate': 0.5, 'min_impurity_decrease': 0.1}
time:  0.333147
0 0.8177626303127506 0.9145252975201117 0.8217821782178217 0.915702479338843 0.7906976744186046 0.9044943820224719 0.7906976744186046 0.9044943820224719 0.7906976744186046 0.9044943820224719 GradientBoostingClassifier {'max_depth': 5, 'loss': 'exponential', 'n_estimators': 10, 'learning_rate': 0.5, 'min_impurity_decrease': 0.1}
Regression:  False
gb
{'max_depth': 5, 'loss': 'exponential', 'n_estimators': 10, 'learning_rate': 1.0, 'min_impurity_decrease': 0}
time:  0.315353
0 0.7897620956963378 0.9054057797575515 0.7953795379537953 0.9066115702479339 0.7637795275590551 0.8934579439252337 0.751937984496124 0.8951310861423221 0.7578125 0.8942937324602432 GradientBoostingClassifier {'max_depth': 5, 'loss': 'exponential', 'n_estimators': 10, 'learning_rate': 1.0, 'min_impurity_decrease': 0}
Regression:  False
gb
{'max_depth': 7, 'loss': 'expon

The max_iter was reached which means the coef_ did not converge


time:  1.399725
0 0.822641005078856 0.9604137579504908 0.8250825082508251 0.959504132231405 0.7878787878787878 0.941712204007286 0.8062015503875969 0.9681647940074907 0.7969348659003832 0.9547553093259464 LogisticRegression {'C': 1, 'penalty': 'l2', 'solver': 'saga'}
Regression:  False
lr
{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}
time:  0.08202
0 0.8293905372894949 0.9626326928617334 0.8316831683168316 0.9619834710743802 0.7954545454545454 0.9468864468864469 0.813953488372093 0.9681647940074907 0.8045977011494252 0.9574074074074075 LogisticRegression {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}
Regression:  False
lr
{'C': 10, 'penalty': 'l2', 'solver': 'saga'}


The max_iter was reached which means the coef_ did not converge


...

time:  68.714051
0 0.6949574151278761 0.5480213794951297 0.4829658088412191 0.300327432383745 0.5378729199566777 0.4225240573501537 0.4612553345445344 0.6760817661883751 MLPRegressor {'hidden_layer_sizes': (50,), 'activation': 'logistic', 'alpha': 0.0001, 'max_iter': 1000, 'solver': 'sgd'}
Regression:  True
rf
{'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 4, 'min_impurity_decrease': 0, 'n_estimators': 200}
time:  14.412593
0 0.7084141981766313 0.46528196517616033 0.5018506761782395 0.21648730711818967 0.5427721578187503 0.352395446319286 0.4401894094844049 0.7665075560771377 RandomForestRegressor {'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 4, 'min_impurity_decrease': 0, 'n_estimators': 200}
Regression:  True
rf
{'max_depth': None, 'min_samples_split': 5, 'min_samples_leaf': 4, 'min_impurity_decrease': 0, 'n_estimators': 200}
time:  14.445521
0 0.7084141981766313 0.46528196517616033 0.5018506761782395 0.21648730711818967 0.5427721578187503 0.3523954


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 2186/2186 [06:54<00:00,  5.27it/s]


Regression:  True
rf
{'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 1, 'min_impurity_decrease': 0, 'n_estimators': 100}
time:  126.3301
0 0.7063714537923301 0.3961944111884402 0.49896063073269004 0.1569700114569548 0.5374535936927167 0.3107179424078785 0.4434132330723507 0.8306999514864188 RandomForestRegressor {'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 1, 'min_impurity_decrease': 0, 'n_estimators': 100}
Regression:  True
rf
{'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 2, 'min_impurity_decrease': 0, 'n_estimators': 50}
time:  59.53872
0 0.713722592717735 0.4233705395565403 0.5093999393557258 0.17924261376439607 0.5417726149409862 0.32977383317326764 0.4317682641558187 0.8066778302151367 RandomForestRegressor {'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 2, 'min_impurity_decrease': 0, 'n_estimators': 50}



A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 2186/2186 [07:02<00:00,  5.17it/s]


Regression:  True
gb
{'max_depth': 7, 'loss': 'huber', 'n_estimators': 100, 'learning_rate': 0.1, 'min_impurity_decrease': 0.1}
time:  106.119792
0 0.695062582519727 0.21119004020830962 0.4831119936189922 0.04460123308318743 0.5185864032149906 0.1353573501446385 0.4610922665431265 0.9518953279377198 GradientBoostingRegressor {'max_depth': 7, 'loss': 'huber', 'n_estimators': 100, 'learning_rate': 0.1, 'min_impurity_decrease': 0.1}
Regression:  True
gb
{'max_depth': 7, 'loss': 'huber', 'n_estimators': 100, 'learning_rate': 0.1, 'min_impurity_decrease': 0.2}
time:  86.043823
0 0.6981395670349904 0.2425417529864276 0.48739885505980385 0.05882650194172926 0.5170120434096407 0.1650525377453762 0.4563103053970047 0.9365526603446153 GradientBoostingRegressor {'max_depth': 7, 'loss': 'huber', 'n_estimators': 100, 'learning_rate': 0.1, 'min_impurity_decrease': 0.2}
Regression:  True
gb
{'max_depth': 5, 'loss': 'huber', 'n_estimators': 100, 'learning_rate': 0.1, 'min_impurity_decrease': 0.1}
time